# Heat Flow Equation
The heat flow equation is a partial differential equation that describes the distribution of heat (or variation in temperature) in a given region over time. It is given by:
$$\frac{\partial u}{\partial t} = \alpha^2 \nabla^2 u$$
where $u$ is the temperature distribution, $t$ is time, $\alpha$ is the thermal diffusivity, and $\nabla^2$ is the Laplacian operator. The Laplacian operator is given by:
$$\nabla^2 u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} + \frac{\partial^2 u}{\partial z^2}$$

# Crank Nicholson

The Crank Nicholson method is a finite difference method used to solve parabolic partial differential equations. It is unconditionally stable and second order accurate in time. The method is an implicit method, which means that the solution at the next time step is obtained by solving an equation that contains the solution at the current time step. The method is based on the implicit trapezoidal rule for integration.

The Crank Nicholson method is given by the following equation:

$$u_{i}^{n + 1} - u_{i}^{n} = \int_{t_n}^{t_{n + 1}} \frac{\partial^2 u}{\partial x^2} dt  = \frac{\Delta t}{2} \left[\left( \frac{\partial^{2} u}{\partial x^{2}} \right)_{i}^{n + 1} + \left( \frac{\partial^{2} u}{\partial x^{2}} \right)_{i}^{n} \right]$$
$$ \Longrightarrow u_{i}^{n + 1} = u_{i}^{n} - r \left( u_{i - 1}^{n} + u_{i + 1}^{n} - 2u_{i}^{n} \right) -  r \left( u_{i - 1}^{n + 1} + u_{i + 1}^{n + 1} - 2u_{i}^{n + 1} \right)$$
with $r = \frac{ \alpha^2 \Delta t}{2 \Delta x^2}$.

We get a linear system of equations of the form:
$$ -r u_{i-1}^{n + 1} + 2(1 + r) u_i^{n + 1} - r u_{i + 1}^{n + 1} = 2(1 - r) u_i^n + r (u_{i - 1}^n + u_{i + 1}^n)$$
Which can be formulated as a Matrix equation 
$$A \mathbf{u}^{n + 1} = B\mathbf{u}^{n}$$
where $A, B$ are a tridiagonal matrix.

For $r = 1$ the system simplifies to:
$$ - u_{i-1}^{n + 1} + 4 u_i^{n + 1} -  u_{i + 1}^{n + 1} =u_{i - 1}^n + u_{i + 1}^n$$



For the heat equation, the boundary conditions are usually Dirichlet boundary conditions, which means that the value of the solution is specified at the boundaries. The boundary conditions are incorporated into the linear system of equations by setting the values of the solution at the boundaries to the specified values.

In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve, cgs
from scipy import sparse as ss

# Initial Configuration

In [ ]:
N = 1000
arr = np.zeros(N)

arr[4*N//10: 6*N//10] = 100

left_bc = 10
right_bc = 10

arr[0] = left_bc
arr[-1] = right_bc

In [ ]:
plt.plot(arr)

## Sparse System

In [ ]:
ones = np.ones(N - 2)
ones.shape

In [ ]:
A = diags([-1*ones[:-1], 4*ones, -1*ones[:-1]], [-1, 0, 1]).tocsr()


In [ ]:
print(f"Arraysize: {A.toarray().nbytes/1e6:.2f} MB")
print(f"Sparse Arraysize: {(A.data.nbytes + A.indptr.nbytes + A.indices.nbytes)/1e6:.2f} MB")

In [ ]:
B = diags([ones[:-1], ones[:-1]], [-1, 1])


In [ ]:
t_end = 5000

interior_points = arr[1: -1]

storage = np.zeros(shape=(t_end, N-2))
storage[0] = interior_points

for i in range(1, t_end):
    
    left_side = interior_points[1]
    right_side = interior_points[-2]
    
    
    
    interior_points = B@interior_points
    
    # apply bc
    interior_points[0] = 2*left_bc + left_side
    interior_points[-1] = 2*right_bc + right_side
    
    interior_points = spsolve(A, interior_points)
    
    storage[i] = interior_points
    

In [ ]:

# Create a figure and axis
fig, ax = plt.subplots()

# Set the x-axis limits
ax.set_xlim(0, N)

# Set the y-axis limits
ax.set_ylim(np.min(storage), np.max(storage))

# Create an empty line object
line, = ax.plot([], [], lw=2, ls="None", marker="o", markersize=2, color="black")

# Update function for the animation
def update(frame):
    # Update the line data with the corresponding time step from the storage array
    line.set_data(np.arange(len(storage[frame])), storage[frame])
    return line,

def init(): 
    line.set_data(np.arange(N), arr) 
    return line, 

animation = FuncAnimation(fig, update, frames=range(0, len(storage), 20), blit=True, init_func=init, interval=25)

animation.save('diffusion1d.gif', writer = 'ffmpeg', fps = 60) 
plt.show()


In [ ]:
from matplotlib import colormaps as cm
import matplotlib as mpl#

fig, ax = plt.subplots()

data = storage[-1]
norm = mpl.colors.Normalize(vmin=data.min(), vmax=data.max())
ax.scatter(np.arange(len(data)), data, cmap=cm["coolwarm"], norm=norm, c = data)

# ax.grid()
ax.set_axis_off()
fig.tight_layout()
fig.savefig("thumbnail.svg", dpi = 500,transparent=True)

# 2D Crank Nicholson
To solve the 2D heat equation using the Crank Nicholson method, we discretize the equation in both space and time. The 2D heat equation is given by:
$$\frac{\partial u}{\partial t} = \alpha^2 \nabla^2 u = \alpha^2 \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)$$
By discretizing the equation in space and time, we get:
$$\frac{u_{i, j}^{n + 1} - u_{i, j}^{n}}{\Delta t} = \alpha^2 \left( \frac{u_{i - 1, j}^{n + 1} - 2u_{i, j}^{n + 1} + u_{i + 1, j}^{n + 1}}{\Delta x^2} + \frac{u_{i, j - 1}^{n + 1} - 2u_{i, j}^{n + 1} + u_{i, j + 1}^{n + 1}}{\Delta y^2} \right)$$
Assuming equal spacing in the x and y directions, we get:
\begin{align*}
u_{i, j}^{n + 1} = u_{i, j}^{n} + &r \left( u_{i - 1, j}^{n} - 2u_{i, j}^{n} + u_{i + 1, j}^{n} + u_{i, j - 1}^{n} - 2u_{i, j}^{n} + u_{i, j + 1}^{n} \right)\\
 +&r \left( u_{i - 1, j}^{n + 1} - 2u_{i, j}^{n + 1} + u_{i + 1, j}^{n + 1} + u_{i, j - 1}^{n + 1} - 2u_{i, j}^{n + 1} + u_{i, j + 1}^{n + 1} \right)
\end{align*}
where $r = \frac{\alpha^2 \Delta t}{2 \Delta x^2}$.

In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import cgs
import matplotlib.pyplot as plt

In [ ]:
r = 0.5
N = 500

## Iteration Matrix
The scheme is given by:
$$(1 + 4r) u_{i, j}^{n + 1} - r u_{i - 1, j}^n - r u_{i + 1, j}^n - r u_{i, j - 1}^n - r u_{i, j + 1}^n = r(u_{i, j - 1} + u_{i, j + 1} + u_{i - 1, j} + u_{i + 1, j} - 4u_{i, j})$$

The iteration matrix is given by:
$$A = \begin{bmatrix}
1 + 4r & -r & 0 & \cdots & 0 & -r & \cdots\\
-r & 1 + 4r & -r & \cdots & 0 & 0& \cdots\\
0 & -r & 1 + 4r & \cdots & 0 & 0& \cdots\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots& \cdots\\
-r & 0 & 0 & \cdots & -r & 1 + 4r\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots\\
\end{bmatrix}$$


In [ ]:
diagonals = [(1 + 4*r)* np.ones(N*N), -r*np.ones(N*N - 1), -r*np.ones(N*N -1), -r*np.ones(N*N - N), -r*np.ones(N*N - N)]

In [ ]:
A = diags(diagonals, [0, -1, 1, N, -N], format="csr")

## Initial Configuration and Boundary Conditions

In [ ]:
from skimage.draw import disk

rr, cc = disk((N//2, N//4), N//15)

mask = np.zeros((N, N), dtype = bool)

mask[rr, cc] = 1

rr, cc = disk((N//2, 3*N//4), N//15)

mask[rr, cc] = 1


In [ ]:
plt.imshow(mask)

In [ ]:
values = np.zeros((N, N))

values[mask] = 10

values = values.flatten()

## Inhomogenity 
The inhomogenous can also be given using a Matrix $M$:
$$M = \begin{bmatrix}
- 4r & r & 0 & \cdots & 0 & r & \cdots\\
r & - 4r & r & \cdots & 0 & 0& \cdots\\
0 & r & - 4r & \cdots & 0 & 0& \cdots\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots& \cdots\\
r & 0 & 0 & \cdots & r & - 4r\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots\\
\end{bmatrix}$$
by 
$$A \mathbf{u}^{n + 1} = M\mathbf{u}^{n}$$

In [ ]:
diagonals = [(1-4*r)* np.ones(N*N), r*np.ones(N*N - 1), r*np.ones(N*N -1), r*np.ones(N*N - N), r*np.ones(N*N - N)]

In [ ]:
M = diags(diagonals, [0, -1, 1, -N, N], format = "csr")

In [ ]:
steps = 3000
arr = np.zeros((steps, N, N))
arr[0] = values.reshape((N, N))

for i in range(1, steps):
    values, _ = cgs(A, M@values)
    arr[i] = values.reshape((N, N))

## Use Cupy

In [ ]:
import cupy as cp
from cupyx.scipy.sparse import linalg as cp_linalg
from cupyx.scipy.sparse import csr_matrix

In [ ]:
A_cupy = csr_matrix(A)
M_cupy = csr_matrix(M)

values_cupy = cp.zeros((N, N))
values_cupy[mask] = 10
values_cupy = values_cupy.flatten()

In [ ]:
%%timeit
cgs(A, M@values)

In [ ]:
%%timeit
cp_linalg.cgs(A_cupy, M_cupy@values_cupy)

In [ ]:
steps = 3000
arr = np.zeros((steps, N, N))
arr[0] = values_cupy.reshape((N, N)).get()

for i in range(1, steps):
    values_cupy, _ = cp_linalg.cgs(A_cupy, M_cupy@values_cupy)
    arr[i] = values_cupy.reshape((N, N)).get()

## Visualize the results 

In [ ]:
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, frameon=False)

X, Y = np.meshgrid(np.arange(0, N), np.arange(0, N))

plot = ax.plot_surface(X, Y, arr[0], cmap="coolwarm")

def updatefig(i, plot):
    ax.clear()
    ax.set_zlim(bottom=0, top = np.max(arr[0]))
    ax.set_axis_off()
    # ax.set_pane_color((0, 0, 0))
    plot = ax.plot_surface(X, Y, arr[i], cmap="coolwarm")
    
    return plot, 
    

ani = FuncAnimation(fig, updatefig, frames=range(0, len(arr), 25), blit=True, fargs=(plot, ))

ani.save('diffusion_gpu.gif', writer = 'ffmpeg', fps = 60)